# 0. FUNCOES

In [1]:
def get_rows_columns_map(table_result, blocks_map):
    rows = {}
    for relationship in table_result['Relationships']:
        if relationship['Type'] == 'CHILD':
            for child_id in relationship['Ids']:
                cell = blocks_map[child_id]
                if cell['BlockType'] == 'CELL':
                    row_index = cell['RowIndex']
                    col_index = cell['ColumnIndex']
                    if row_index not in rows:
                        # create new row
                        rows[row_index] = {}

                    # get the text value
                    rows[row_index][col_index] = get_text(cell, blocks_map)
    return rows


def get_text(result, blocks_map):
    text = ''
    if 'Relationships' in result:
        for relationship in result['Relationships']:
            if relationship['Type'] == 'CHILD':
                for child_id in relationship['Ids']:
                    word = blocks_map[child_id]
                    if word['BlockType'] == 'WORD':
                        text += word['Text'] + ' '
                    if word['BlockType'] == 'SELECTION_ELEMENT':
                        if word['SelectionStatus'] == 'SELECTED':
                            text += 'X '
    return text


def get_table_csv_results(file_name):
    with open(file_name, 'rb') as file:
        img_test = file.read()
        bytes_test = bytearray(img_test)
        print('Image loaded', file_name)

    # process using image bytes
    # get the results
    client = boto3.client('textract')

    response = client.analyze_document(Document={'Bytes': bytes_test}, FeatureTypes=['TABLES'])

    # Get the text blocks
    blocks = response['Blocks']
    pprint(blocks)

    blocks_map = {}
    table_blocks = []
    for block in blocks:
        blocks_map[block['Id']] = block
        if block['BlockType'] == "TABLE":
            table_blocks.append(block)

    if len(table_blocks) <= 0:
        return "<b> NO Table FOUND </b>"

    csv = ''
    for index, table in enumerate(table_blocks):
        csv += generate_table_csv(table, blocks_map, index + 1)
        csv += '\n\n'

    return csv


def generate_table_csv(table_result, blocks_map, table_index):
    rows = get_rows_columns_map(table_result, blocks_map)

    table_id = 'Table_' + str(table_index)

    # get cells.
    csv = 'Table: {0}\n\n'.format(table_id)

    for row_index, cols in rows.items():

        for col_index, text in cols.items():
            csv += '{}'.format(text) + ","
        csv += '\n'

    csv += '\n\n\n'
    return csv

# 1. INSTALL e LOGIN AWS

In [2]:
!pip install pdf2img
!pip install PyPDF2
!apt-get install -qq poppler-utils
!pip install boto3
!pip install layoutparser torchvision && pip install "detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.5#egg=detectron2"

  Preparing metadata (setup.py) ... done
  Created wheel for pdf2img: filename=pdf2img-0.1.2-py3-none-any.whl size=9241 sha256=e7720546d3499c4147ddc08bad9790ef49627e243da1f1d8b6c45b3c576b5c57
  Stored in directory: /root/.cache/pip/wheels/7f/ce/db/9cb36b5b9740928098b2a9784653023a09346063988de88907
Successfully built pdf2img
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.8 MB/s eta 0:00:00
Selecting previously unselected package poppler-utils.
(Reading database ... 123069 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.86.1-0ubuntu1.1_amd64.deb ...
Unpacking poppler-utils (0.86.1-0ubuntu1.1) ...
Setting up poppler-utils (0.86.1-0ubuntu1.1) ...
Processing triggers for man-db (2.9.1-1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.9/135.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━

In [ ]:
# amazon CLI installation
!curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
!unzip awscliv2.zip
!sudo ./aws/install
!aws --version

In [ ]:
# amazon login (keys) - digitar no terminal sem "#!"
#!aws configure
#region_name: us-east-2

# 2. IMPORTS

In [ ]:
# imports
import sys
import pdf2image
import PyPDF2
import ast
import os
import boto3
import webbrowser
import json
import io

import pandas       as pd
import numpy        as np
import layoutparser as lp

from io           import BytesIO
from pprint       import pprint
from google.colab import drive

In [ ]:
# read from google drive files
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 3. MULTIPLE DETECTION SCORE (not running)

In [ ]:
# path_import = "/content/drive/MyDrive/Colab Notebooks/pdti/data/"

# list_pdf_files = ['PDTIC20212023-agu.pdf', 'PDTIC-2019-2022-MInfra.pdf', 'PDTIC-2022a2024-datasus.pdf']

# dict_models = {'{0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"}': ['lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config'],
#                '{1:"TextRegion", 2:"ImageRegion", 3:"TableRegion", 4:"MathsRegion", 5:"SeparatorRegion", 6:"OtherRegion"}':['lp://PrimaLayout/mask_rcnn_R_50_FPN_3x/config'],
#                '{0: "Table"}':['lp://TableBank/faster_rcnn_R_101_FPN_3x/config']}

In [ ]:
# path = "/content/drive/MyDrive/"
# score_total = {}

# # para cada modelo
# for k, v in dict_models.items():
#   for catalog in v:
#     print(f'##############################\nMODELO: {catalog}\n##############################')

#     # armazene na variável model
#     model = lp.Detectron2LayoutModel(catalog,
#                                  extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.5],
#                                  label_map=ast.literal_eval(k))

#     # crie uma pasta com o nome do modelo
#     folder_name = catalog.translate({ord('/'): '_', ord(':'): '_'})
#     os.makedirs(path+folder_name)



#     # para cada arquivo PDF
#     for arquivo_pdf in list_pdf_files:
#       print(f'##############################\nARQUIVO: {arquivo_pdf}\n##############################')

#       # crie uma pasta com o nome do arquivo PDF
#       os.makedirs(f'{path + folder_name}/{arquivo_pdf[:-4]}')

#       # armazene a quantidade de páginas
#       file_ = open(path_import + arquivo_pdf, 'rb')
#       readpdf = PyPDF2.PdfReader(file_)
#       total_pages = len(readpdf.pages)

#       # reseta score do arquivo
#       score_arquivo = 0
#       soma_score_paginas = 0

#       # para cada página
#       for page in range(0, total_pages):
#         print(f'### Analisando página: {page} ###')

#         # importa pdf
#         img = np.asarray(pdf2image.convert_from_path(path_import + arquivo_pdf)[page])

#         # detecta layout
#         layout_result = model.detect(img)

#         # detecta blocos com tabelas
#         table_blocks = lp.Layout([b for b in layout_result if b.type in ['Table', 'TableRegion']]) # selecione o layout das tabelas
#         lista_score = table_blocks.get_info('score') # armazene a lista de scores

#         # score por página
#         if len(lista_score) > 0:  # se a lista de scores estiver preenchida
#           score_pagina = (sum(lista_score) / len(lista_score)) # armazene a média de score
#           print(f'### Score página {page}: {score_pagina} ###\n\n')
#           soma_score_paginas += score_pagina

#         # output img
#         img_out = lp.draw_box(img, layout_result,  box_width=5, box_alpha=0.2, show_element_type=True)
#         img_out.save(f"{path + folder_name}/{arquivo_pdf[:-4]}/{page}.jpeg" )

#       # score do arquivo PDF
#       score_arquivo = soma_score_paginas/total_pages

#       # armazene o score do arquivo em um dicionário
#       score_total[f'{catalog} || {arquivo_pdf}'] = score_arquivo

# df = pd.DataFrame.from_dict(score_total, orient='index')

In [ ]:
# df

,0
lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config || PDTIC20212023-agu.pdf,0.380221
lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config || PDTIC-2019-2022-MInfra.pdf,0.488772
lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config || PDTIC-2022a2024-datasus.pdf,0.275569
lp://PrimaLayout/mask_rcnn_R_50_FPN_3x/config || PDTIC20212023-agu.pdf,0.293111
lp://PrimaLayout/mask_rcnn_R_50_FPN_3x/config || PDTIC-2019-2022-MInfra.pdf,0.577851
lp://PrimaLayout/mask_rcnn_R_50_FPN_3x/config || PDTIC-2022a2024-datasus.pdf,0.234143
lp://TableBank/faster_rcnn_R_101_FPN_3x/config || PDTIC20212023-agu.pdf,0.581732
lp://TableBank/faster_rcnn_R_101_FPN_3x/config || PDTIC-2019-2022-MInfra.pdf,0.479744
lp://TableBank/faster_rcnn_R_101_FPN_3x/config || PDTIC-2022a2024-datasus.pdf,0.344303


In [ ]:
# df.to_csv("/content/drive/MyDrive/Colab Notebooks/pdti/output/score.csv", sep=',')

# 4. CROP (not running)

In [ ]:
# path_import = "/content/drive/MyDrive/Colab Notebooks/pdti/data/"

# list_pdf_files = ['PDTIC-2019-2022-MInfra.pdf']

# dict_models = {'{0: "Table"}':['lp://TableBank/faster_rcnn_R_101_FPN_3x/config']}

# path = "/content/drive/MyDrive/"

# # score_total = {}

# # para cada modelo
# for k, v in dict_models.items():
#   for catalog in v:
#     print(f'##############################\nMODELO: {catalog}\n##############################')

#     # armazene na variável model
#     model = lp.Detectron2LayoutModel(catalog,
#                                  extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.5],
#                                  label_map=ast.literal_eval(k))

#     # crie uma pasta com o nome do modelo
#     folder_name = catalog.translate({ord('/'): '_', ord(':'): '_'})
#     os.makedirs(path+folder_name)



#     # para cada arquivo PDF
#     for arquivo_pdf in list_pdf_files:
#       print(f'##############################\nARQUIVO: {arquivo_pdf}\n##############################')

#       # crie uma pasta com o nome do arquivo PDF
#       os.makedirs(f'{path + folder_name}/{arquivo_pdf[:-4]}')

#       # armazene a quantidade de páginas
#       file_ = open(path_import + arquivo_pdf, 'rb')
#       readpdf = PyPDF2.PdfReader(file_)
#       total_pages = len(readpdf.pages)

#       # reseta score do arquivo
#       score_arquivo = 0
#       soma_score_paginas = 0

#       # para cada página
#       for page in range(0, total_pages):
#         print(f'### Analisando página: {page} ###')

#         # importa pdf
#         img = np.asarray(pdf2image.convert_from_path(path_import + arquivo_pdf)[page])
#         img_crop = pdf2image.convert_from_path(path_import + arquivo_pdf)[page]

#         # detecta layout
#         layout_result = model.detect(img)

#         # detecta blocos com tabelas
#         table_blocks = lp.Layout([b for b in layout_result if b.type=='Table']) # selecione o layout das tabelas

#         # lista de blocos
#         list_block = table_blocks.get_info('block')

#         # para cada bloco retorne as coordenadas
#         count = 0
#         for bloco in list_block:
#           coord = bloco.coordinates
#           croped_img = img_crop.crop(coord)
#           croped_img.save(f"{path + folder_name}/{arquivo_pdf[:-4]}/{page}.{count}.jpeg" )
#           count += 1

# 5. TEXTRACT

In [ ]:
path_docs = "/content/drive/MyDrive/lp___TableBank_faster_rcnn_R_101_FPN_3x_config"
path = "/content/drive/MyDrive/"

docs=[]

# registra pastas
for filename in os.listdir(path_docs):
  docs.append(filename)

# para cada doc PDF
for doc in docs:

  # crie uma pasta com o nome do PDF (PDF_tables)
  folder_name = f'{doc}_TABELAS'
  os.makedirs(path+folder_name)

  # para cada imagem (croped)
  for file_croped in os.listdir(f'{path_docs}/{doc}'):

    table_csv = get_table_csv_results(f'{path_docs}/{doc}/{file_croped}')

    output_file = file_croped.translate({ord('.'): '_'})[:-5]

    # gere um csv
    with open(f'{path}/{folder_name}/{output_file}.csv', "wt") as fout:
        fout.write(table_csv)